In [2]:
# here, we combine energy_use.csv and energy_production.csv into the proper timer series format and output energy_final.csv

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df_use = pd.read_csv('../../SharedData/dataset-generation/energy_use.csv')
df_prod = pd.read_csv('../../SharedData/dataset-generation/energy_production.csv')

d = {'state':[], 'date':[], 'monthly_energy_prod':[], 'monthly_energy_use':[], 'yearly_energy_prod':[], 'yearly_energy_use':[]}
new_df = pd.DataFrame(data=d)

In [3]:
abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

In [4]:
df_use = df_use[df_use['State'].isin(abbreviations)]
df_prod = df_prod[df_prod['State'].isin(abbreviations)]

In [5]:
df_use.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AK,"59,303","70,020","76,642","78,488","82,793","85,319","100,481","112,625","119,992",...,"587,460","586,053","602,906","582,922","590,212","593,292","595,231","629,029","688,282","724,059"
1,AL,"842,283","806,682","853,424","885,811","949,411","1,011,125","1,055,347","1,100,904","1,208,038",...,"1,882,362","1,929,314","1,880,544","1,895,744","1,867,484","1,913,464","1,883,941","1,793,090","1,924,516","1,902,374"
2,AR,"417,153","423,192","446,994","474,761","515,874","515,759","553,146","581,814","620,382",...,"1,092,848","1,108,345","1,043,455","1,036,998","1,043,543","1,103,282","1,078,044","1,010,754","1,054,467","1,052,517"
3,AZ,"273,431","294,660","313,581","337,213","357,961","362,858","390,820","400,280","442,172",...,"1,419,110","1,422,839","1,439,724","1,462,217","1,462,699","1,474,472","1,493,085","1,435,329","1,481,293","1,526,882"
4,CA,"3,360,697","3,513,822","3,623,382","3,825,677","4,112,125","4,267,121","4,525,141","4,703,411","4,979,463",...,"7,162,622","7,073,527","7,169,881","7,168,226","7,176,823","7,317,326","7,258,664","6,462,830","6,808,107","6,882,442"


In [6]:
# loop through each year in 1960-2022 in given row in dataframe
# copy each year's energy data 12 times (one for each month) into 12 rows of new_df
# for a total of 12*63*50 entries

month_counter = 1   # acts like modulo 12 arithmetic
idx = 0             # keeps track of row index

for state in range(50):
    for year in range(1,len(df_prod.iloc[state])):
        for month in range(12):
            if month_counter < 12:   
                new_df.loc[idx] = [df_prod.iloc[state][0],
                                   df_prod.columns[year]+'-'+'{:02d}'.format(month_counter),
                                   None,
                                   None,
                                   df_prod.iloc[state][year], 
                                   df_use.iloc[state][year]]     
                month_counter += 1
            else:
                new_df.loc[idx] = [df_prod.iloc[state][0],
                    df_prod.columns[year]+'-'+'{:02d}'.format(month_counter),
                    int(df_prod.iloc[state][year].replace(',', ''))/12, 
                    int(df_use.iloc[state][year].replace(',', ''))/12,
                    df_prod.iloc[state][year], 
                    df_use.iloc[state][year]]     
                month_counter = 1
            idx += 1

In [7]:
# clean up data by removing commas and converting to int

new_df['yearly_energy_use'] = new_df['yearly_energy_use'].str.replace(',', '')
new_df['yearly_energy_use'] = new_df['yearly_energy_use'].astype(int)

new_df['yearly_energy_prod'] = new_df['yearly_energy_prod'].str.replace(',', '')
new_df['yearly_energy_prod'] = new_df['yearly_energy_prod'].astype(int)

In [8]:
# linear interpolation to fill in months based on end-of-year data
# and create yearly_gdp_rel_2017 column

for state in abbreviations:
    new_df.loc[new_df.state == state,'monthly_energy_prod']=new_df[new_df.state == state].monthly_energy_prod.interpolate(method='linear')
    new_df.loc[new_df.state == state,'monthly_energy_use']=new_df[new_df.state == state].monthly_energy_use.interpolate(method='linear')


In [9]:
new_df['monthly_energy_flow'] = new_df['monthly_energy_prod']-new_df['monthly_energy_use']
new_df['yearly_energy_flow'] = new_df['yearly_energy_prod']-new_df['yearly_energy_use']

In [10]:
# export new_df to a .csv
new_df.to_csv('../../SharedData/dataset-generation-final/energy_final.csv',index=False)